In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib3
import certifi
import re
from collections import defaultdict
from copy import copy, deepcopy

In [3]:
http = urllib3.PoolManager(
     cert_reqs='CERT_REQUIRED',
     ca_certs=certifi.where())

In [239]:
numery = []

with open('numery_mini_ang.txt') as f:
    numery = [int(n) for n in f.read().splitlines()]

with open('numery_mini.txt') as f:
    numery = [int(n) for n in f.read().splitlines()]
    
with open('numery_lwow.txt') as f:
    numery = numery + [int(n) for n in f.read().splitlines()]
    
with open('numery_mim.txt') as f:
    numery = numery + [int(n) for n in f.read().splitlines()]

In [5]:
def link_from_num(idd):
    return "http://mathgenealogy.org/id.php?id=" + str(idd)

In [49]:
# testing

for idd in numery:
    link = link_from_num(idd)
    page = http.request('GET', link, preload_content=False)
    soup = BeautifulSoup(page.read(), 'html.parser')
    mat_name = soup.find('h2')
    name = str(mat_name.string)
    st = mat_name.fetchNextSiblings()
    print(idd)
    ss = str(soup.select("#paddingWrapper > div:nth-child(6) > span"))
    print(re.search(r".*([1-3][0-9]{3})", ss).group(1))
    school = soup.select_one("#paddingWrapper > div:nth-child(6) > span > span")
    print(school.string)
    break

155901
2010
Politechnika Warszawska


In [240]:
def scrap_id(idd):
    link = link_from_num(idd)
    page = http.request('GET', link, preload_content=False)
    soup = BeautifulSoup(page.read(), 'html.parser')
    mat_name = soup.find('h2')
    try:
        name = str(mat_name.string).strip()
        st = mat_name.fetchNextSiblings()
        name_s = soup.select("#paddingWrapper > div:nth-child(6) > span")
        year = re.search(r".*([1-3][0-9]{3})", str(name_s))
        if year is not None:
            year = year.group(1)
        else:
            year = -1
            print("No year for:", name, idd)
        school = soup.select_one("#paddingWrapper > div:nth-child(6) > span > span").string
    except Exception as exc:
        print(exc)
        print(idd)
        print(mat_name)
        print(soup)
    advisors = []
    for e in st:
        if str(e).find("Advisor") != -1:
            res = re.search(r"id=(\d*)", str(e))
            try:
                advisors.append(int(res.group(1)))
            except Exception as exc:
                if str(e).find("Unknown") != -1:
                    advisors.append(-1)
                else:
                    print("ERRRRROR")
                    print(exc)
                    print(e)
                    print(idd)
    d = dict()
    d['advisors'] = copy(advisors)
    d['name'] = name
    d['school'] = str(school)
    d['year'] = int(year)
    return d

In [241]:
db = defaultdict(dict)

for idd in numery:
    db[idd] = deepcopy(scrap_id(idd))

No year for: Krystyna  Dzierzbicka-Mazaraki 134439
No year for: Karol  Krzyżewski 69809
No year for: Jerzy  Muszyński 134440
No year for: Zofia  Adamowicz 22613
No year for: Magdy Yousef Barsoom 134438
No year for: Andrzej Tadeusz Birkholc 86232
No year for: Ryszard  Bittner 86237
No year for: Tomasz  Bojdecki 86240
No year for: Maciej  Brynski 22605
No year for: Malgorzata  Dubiel 22614
No year for: Andrzej  Ehrenfeucht 22604
No year for: Lej Tsin Gan 86231
No year for: Benjamin  Goldys 68298
No year for: Jacek  Graczyk 127971
No year for: Wojciech  Hyb 127976
No year for: Andrzej  Janiczak 22603
No year for: Witold  Kolodziej 86243
No year for: Stanislaw  Krajewski 22612
No year for: Michal  Krynicki 22610
No year for: Włodzimierz  Lesisz 81438
No year for: Wan Dzien Lu 86230
No year for: Janusz  Onyszkiewicz 22606
No year for: Jacek   Pawelczyk 129108
No year for: Stefan  Pokorski 129125
No year for: Maria K. Pospieszalska 134441
No year for: Maciej  Skwarczyński 60565
No year for: 

In [61]:
#db_hist = deepcopy(db)

In [9]:
#db = deepcopy(db_hist)

In [242]:
depth = 10
db_last = db
db_new = defaultdict(dict)
for i in range(depth):
    for k, d in db_last.items():
        for adv_id in d['advisors']:
            if not adv_id in db and not adv_id in db_new and adv_id != -1:
                db_new[adv_id] = deepcopy(scrap_id(adv_id))
    db = {**db, **db_new}
    db_last = deepcopy(db_new)

No year for: Andrzej  Wierzbicki 97165
No year for: Tadeusz  Kaczorek 75005
No year for: Adam  Idzik 195202
No year for: Tadeusz  Traczyk 75416
No year for: Władysław  Findeisen 31086
No year for: Jacek  Szymanowski 174151
No year for: Urzyczyn  Paweł 228542
No year for: Wojciech  Wojtynski 20830
No year for: Jozef  Spalek 55675
No year for: Wojciech  Niemiro 151767
No year for: Wieslaw  Szlenk 74463
No year for: Leslaw W. Szczerba 35198
No year for: Piotr  Gwiazda 179958
No year for: Jerzy  Browkin 211233
No year for: Marek  Niezgódka 117786
No year for: Henryk  Greniewski 114616
No year for: Władysław  Natanson 106278
No year for: Czeslaw  Bessaga 37038
No year for: Pierre  Priouret 77999
No year for: César  Macchi 191473
No year for: Aleksei Nikolaevich Krylov 107558
No year for: Jacques  Deny 114298
No year for: Christopher S. Strachey 75007
No year for: Sagdy Khasanovich Sirazhdinov 17543
No year for: Vladimir E. Alekseev 153907
No year for: Osip Ivanovich Somov 519
No year for: B

In [12]:
import networkx as nx
import matplotlib.pyplot as plt
import pygraphviz as pgv
import pickle

In [264]:
graph = nx.DiGraph()

In [265]:
for mat_id, mat in db.items():
    for adv in mat['advisors']:
        if adv != -1:
            g_tmp = graph
            graph.add_edge(adv, mat_id)
            if nx.is_forest(graph) == False:
                graph.remove_edge(adv, mat_id)
            

In [245]:
graph_lab = nx.relabel_nodes(graph, {mat_id : mat['name'] for mat_id, mat in db.items()})

In [261]:
col_converter = {
    'Politechnika Warszawska' : 'red',
    'Technical University of Warsaw' : 'red',
    'Uniwersytet Warszawski' : 'green',
    'University of Lwów' : 'blue'
}

In [288]:
G = nx.nx_agraph.to_agraph(graph)
for e in G:
    ie = int(e)
    if ie in db:
        e.attr['color']= col_converter.get(db[ie]['school'], 'black')
        e.attr['label'] = db[ie]['name']
    
#G.node_attr['color']='blue'
G.layout(prog='fdp')
G.draw(format='svg', path='g.svg')

In [277]:
res_db = defaultdict(int)
for idd, mat in db.items():
    for adv in mat['advisors']:
        if adv in db:
            if mat['school'] != db[adv]['school']:
                direction = db[adv]['school'] + " -> " + mat['school']
                res_db[direction] += 1

In [287]:
[(name, count) for name, count in res_db.items() if count > 4] 

[('Instytut Podstawowych Problemów Techniki PAN -> Politechnika Warszawska',
  10),
 ('None -> Politechnika Warszawska', 11),
 ('Universität Wien -> University of Lwów', 5),
 ('Polska Akademia Nauk -> Uniwersytet Warszawski', 10),
 ('Institute of Mathematics, Polish Academy of Sciences -> Uniwersytet Warszawski',
  25),
 ('University of Lwów -> Uniwersytet Warszawski', 28),
 ('Uniwersytet Jagielloński -> Uniwersytet Warszawski', 12),
 ('Lomonosov Moscow State University -> Uniwersytet Warszawski', 10),
 ('University of California, Berkeley -> Uniwersytet Warszawski', 17),
 ('Ivane Javakhishvili Tbilisi State University -> Uniwersytet Warszawski',
  16),
 ('Uniwersytet Marii Curie-Skłodowskiej -> Uniwersytet Warszawski', 5),
 ('None -> Uniwersytet Warszawski', 11),
 ('Uniwersytet im. Adama Mickiewicza w Poznaniu -> Uniwersytet Warszawski', 5)]

In [296]:
def get_mat_json(graph, idd, db):
    reachable_nodes = nx.descendants(graph, idd)
    reachable_nodes.add(idd)
    res_g = nx.relabel_nodes(nx.subgraph(graph, reachable_nodes), {mat_id : mat['name'] for mat_id, mat in db.items()})
    
    return nx.tree_data(res_g , db[idd]['name'])

In [297]:
get_mat_json(graph, 12681, db)

{'id': 'Stefan  Banach',
 'children': [{'id': 'Stanislaw  Mazur',
   'children': [{'id': 'Maciej  Altman'},
    {'id': 'Andrzej Tadeusz Birkholc'},
    {'id': 'Ryszard  Bittner'},
    {'id': 'Tomasz  Bojdecki'},
    {'id': 'Lej Tsin Gan'},
    {'id': 'Witold  Kolodziej'},
    {'id': 'Tadeusz Jan Lezanski'},
    {'id': 'Wan Dzien Lu'},
    {'id': 'Hanna  Marcinkowska'},
    {'id': 'Krzysztof  Maurin',
     'children': [{'id': 'Janusz  Czyż'},
      {'id': 'Jan  Rogulski'},
      {'id': 'Krzysztof  Gawędzki'},
      {'id': 'Jerzy   Kijowski',
       'children': [{'id': 'Jacek  Jezierski',
         'children': [{'id': 'Witold  Chmielowiec'}]}]},
      {'id': 'Jacek  Komorowski'},
      {'id': 'Wojciech  Lisiecki'},
      {'id': 'Kazimierz  Napiórkowski',
       'children': [{'id': 'Slawomir  Klimek'}]},
      {'id': 'Paweł  Urbański'},
      {'id': 'Stanisław Lech Woronowicz',
       'children': [{'id': 'Paweł Łukasz Kasprzak'},
        {'id': 'Wiesław  Pusz'},
        {'id': 'Piotr Mikoł

In [299]:
for i, e in enumerate(db.values()):
    print(e['name'], e['school'])
    if i > 10:
        break

Agnieszka  Badeńska Politechnika Warszawska
Czesław  Bagiński Politechnika Warszawska
Krystian Waldemar Baniak Politechnika Warszawska
Konstancja  Bobecka Politechnika Warszawska
Nadzieja  Borowik Politechnika Warszawska
T.  Brengos Politechnika Warszawska
Włodzimierz  Bryc Politechnika Warszawska
Joanna  Chachulska Politechnika Warszawska
Konrad  Ciecierski Politechnika Warszawska
Janusz  Czyż Politechnika Warszawska
Anna  Dembinska Politechnika Warszawska
Asen Lubomirov Dontchev Politechnika Warszawska
